In [1]:
import pandas as pd

TRAIN = '/kaggle/input/digit-recognizer/train.csv'
train_df = pd.read_csv(filepath_or_buffer=TRAIN, )
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
from warnings import filterwarnings
from plotly import express
filterwarnings(action='ignore', category=UserWarning)

express.histogram(data_frame=train_df, x='label',).show()
express.pie(data_frame=train_df, names='label', color='label').show()

In [3]:
from arrow import now
from umap import UMAP

time_start = now()
columns = [column for column in train_df.columns if column.startswith('pixel')]
umap = UMAP(random_state=2024, verbose=True, n_jobs=1, low_memory=False, n_epochs=1000)
train_df[['x', 'y']] = pd.DataFrame(data=umap.fit_transform(X=train_df[columns]))
express.scatter(data_frame=train_df, x='x', y='y', color='label').show()
print('done with UMAP in {}'.format(now() - time_start))

2024-03-28 19:06:38.817468: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 19:06:38.817609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 19:06:38.968032: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


UMAP(low_memory=False, n_epochs=1000, n_jobs=1, random_state=2024, verbose=True)
Thu Mar 28 19:06:54 2024 Construct fuzzy simplicial set
Thu Mar 28 19:06:54 2024 Finding Nearest Neighbors
Thu Mar 28 19:06:54 2024 Building RP forest with 15 trees
Thu Mar 28 19:07:04 2024 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
	Stopping threshold met -- exiting after 5 iterations
Thu Mar 28 19:07:32 2024 Finished Nearest Neighbor Search
Thu Mar 28 19:07:37 2024 Construct embedding


Epochs completed:   0%|            0/1000 [00:00]

	completed  0  /  1000 epochs
	completed  100  /  1000 epochs
	completed  200  /  1000 epochs
	completed  300  /  1000 epochs
	completed  400  /  1000 epochs
	completed  500  /  1000 epochs
	completed  600  /  1000 epochs
	completed  700  /  1000 epochs
	completed  800  /  1000 epochs
	completed  900  /  1000 epochs
Thu Mar 28 19:10:09 2024 Finished embedding


done with UMAP in 0:03:15.809692


Four of our digits are for the most part in isolated clusters; the other six digits are in clusters that have regions of ambiguity.

Let's separate out a validation set and use logistic regression to set a baseline for our model.

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from arrow import now

time_start = now()
target = 'label'
X_train, X_test, y_train, y_test = train_test_split(train_df[columns], train_df[target], test_size=0.05, random_state=2024, stratify=train_df[target])

regression = LogisticRegression(max_iter=400, tol=1e-6, verbose=1)
regression.fit(X=X_train, y=y_train)
print('fit complete after {} iterations.'.format(regression.n_iter_[0]))
print('accuracy: {:5.4f}'.format(regression.score(X=X_test, y=y_test)))
print('done in {}'.format(now() - time_start))

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         7850     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.18731D+04    |proj g|=  6.43202D+05


 This problem is unconstrained.



At iterate   50    f=  9.96410D+03    |proj g|=  1.20240D+04

At iterate  100    f=  8.86072D+03    |proj g|=  4.02691D+03

At iterate  150    f=  8.36002D+03    |proj g|=  6.65632D+03

At iterate  200    f=  8.04345D+03    |proj g|=  4.16573D+03

At iterate  250    f=  7.81119D+03    |proj g|=  4.17774D+03

At iterate  300    f=  7.64330D+03    |proj g|=  2.73778D+03

At iterate  350    f=  7.51623D+03    |proj g|=  2.34170D+03

At iterate  400    f=  7.41238D+03    |proj g|=  2.55793D+03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 7850    400    435      1     0     0   2.558D+03   7.412D+03
  F =   7412.37782459

In [5]:
from arrow import now
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

MODEL = {
    'Neural Net 1.0': MLPClassifier(alpha=1, max_iter=2000, random_state=2024),
}

# we have tried k in {3, 5} here; 3 is best so far
for n_neighbors in range(2, 12):
    MODEL['{} Nearest Neighbors'.format(n_neighbors)] = KNeighborsClassifier(n_neighbors=n_neighbors)

for n_estimators in range(5, 100, 5):
    MODEL['{} estimator Random Forest'.format(n_estimators)] = RandomForestClassifier(n_estimators=n_estimators,  random_state=2024)

result = []
for name, clf in MODEL.items():
    time_start = now()
    clf.fit(X=X_train, y=y_train)
    score = clf.score(X=X_test, y=y_test)
    result.append((score, name))
    print('{:5.4f} {} {}'.format(score, now() - time_start, name))
result = sorted(result, key=lambda x: x[0], reverse=True)
print('best: {:5.4f} {}'.format(result[0][0], result[0][1]))

0.9614 0:02:43.232577 Neural Net 1.0
0.9633 0:00:04.143781 2 Nearest Neighbors
0.9714 0:00:04.033893 3 Nearest Neighbors
0.9695 0:00:04.030552 4 Nearest Neighbors
0.9724 0:00:04.264851 5 Nearest Neighbors
0.9724 0:00:04.091779 6 Nearest Neighbors
0.9710 0:00:04.146535 7 Nearest Neighbors
0.9710 0:00:04.291211 8 Nearest Neighbors
0.9710 0:00:04.544956 9 Nearest Neighbors
0.9695 0:00:04.251140 10 Nearest Neighbors
0.9700 0:00:04.073806 11 Nearest Neighbors
0.9129 0:00:02.005527 5 estimator Random Forest
0.9452 0:00:03.826217 10 estimator Random Forest
0.9543 0:00:05.117645 15 estimator Random Forest
0.9557 0:00:07.060243 20 estimator Random Forest
0.9610 0:00:09.427304 25 estimator Random Forest
0.9624 0:00:10.915528 30 estimator Random Forest
0.9629 0:00:12.843940 35 estimator Random Forest
0.9657 0:00:14.188753 40 estimator Random Forest
0.9657 0:00:15.582388 45 estimator Random Forest
0.9657 0:00:17.288755 50 estimator Random Forest
0.9671 0:00:19.457286 55 estimator Random Forest
0.9

Let's make our submission.

This is our best submission so far; it's cheap to generate so let's generate it every time.

In [6]:
import arrow

TEST = '/kaggle/input/digit-recognizer/test.csv'

time_start = arrow.now()
best = KNeighborsClassifier(n_neighbors=3)
best.fit(X=train_df[columns], y=train_df[target])

best_df = pd.DataFrame(data=best.predict(X=pd.read_csv(filepath_or_buffer=TEST, ))).reset_index()
best_df.columns = ['ImageId', 'Label']
best_df['ImageId'] = best_df['ImageId'] + 1
best_file = '/kaggle/working/best.csv.zip'
print('{} : writing best so far guess to {}'.format(arrow.now() - time_start, best_file))
best_df.to_csv(path_or_buf=best_file, index=False, compression='zip')
print('{} : done.'.format(arrow.now() - time_start, ))

0:00:56.350049 : writing best so far guess to /kaggle/working/best.csv.zip
0:00:56.430866 : done.


In [7]:
import arrow

TEST = '/kaggle/input/digit-recognizer/test.csv'

time_start = arrow.now()
current = RandomForestClassifier(n_estimators=45,  random_state=2024).fit(X=train_df[columns], y=train_df[target])

result_df = pd.DataFrame(data=current.predict(X=pd.read_csv(filepath_or_buffer=TEST, ))).reset_index()
result_df.columns = ['ImageId', 'Label']
result_df['ImageId'] = result_df['ImageId'] + 1
output_file = '/kaggle/working/submission.csv.zip'
print('{} : writing submission to {}'.format(arrow.now() - time_start, output_file))
result_df.to_csv(path_or_buf=output_file, index=False, compression='zip')
print('{} : done.'.format(arrow.now() - time_start, ))

0:00:19.822941 : writing submission to /kaggle/working/submission.csv.zip
0:00:19.904434 : done.


In [8]:
test_df = pd.read_csv(filepath_or_buffer=TEST, )
test_df[['x', 'y']] = pd.DataFrame(data=umap.fit_transform(X=test_df))
test_df['Label'] = result_df['Label'].tolist()
express.scatter(data_frame=test_df, x='x', y='y', color='Label').show()

UMAP(low_memory=False, n_epochs=1000, n_jobs=1, random_state=2024, verbose=True)
Thu Mar 28 19:21:23 2024 Construct fuzzy simplicial set
Thu Mar 28 19:21:24 2024 Finding Nearest Neighbors
Thu Mar 28 19:21:24 2024 Building RP forest with 13 trees
Thu Mar 28 19:21:26 2024 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
	Stopping threshold met -- exiting after 5 iterations
Thu Mar 28 19:21:31 2024 Finished Nearest Neighbor Search
Thu Mar 28 19:21:31 2024 Construct embedding


Epochs completed:   0%|            0/1000 [00:00]

	completed  0  /  1000 epochs
	completed  100  /  1000 epochs
	completed  200  /  1000 epochs
	completed  300  /  1000 epochs
	completed  400  /  1000 epochs
	completed  500  /  1000 epochs
	completed  600  /  1000 epochs
	completed  700  /  1000 epochs
	completed  800  /  1000 epochs
	completed  900  /  1000 epochs
Thu Mar 28 19:23:10 2024 Finished embedding
